In this notebook we create few plots of wind intensity measured by satellite, with a particular interest on the Canary Islands, which frequently generate the so-called *von Karman* vortex street.

In [1]:
import datetime
import re
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import filament
import logging
import copy
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
from importlib import reload
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting")

INFO:root:Starting


## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [3]:
year, month, day = 2019, 7, 10
figdir = "../figures/wind/"
visibledir = "/data/Visible/"
os.path.isdir(visibledir)

True

## Data

In [4]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

### Visible image

In [5]:
#visibleim = datewind.strftime("aqua-%Y-%m-%dT00_00_00Z.tiff")
#visibleim = datewind.strftime("Sentinel-3_OLCI_%Y-%m-%d.tiff")
visibleim = datewind.strftime("Sentinel-2_L1C_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Terra-%Y-%m-%dT00_00_00Z.tiff")
if os.path.exists(os.path.join(visibledir, visibleim)):
    print("Reading geoTIFF file {}".format(visibleim))
    vis = filament.Visible()
    vis.read_geotiff(os.path.join(visibledir, visibleim))
else:
    logger.warn("No visible image, specify domain coordinates")

Reading geoTIFF file Sentinel-2_L1C_2019-07-10.tiff


In [6]:
extent2 = copy.deepcopy(vis.extent)
extent2[0] -= 360.
extent2[1] -= 360.

### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

With these pieces of information we can create a list of files URLs.

In [7]:
filelist1 = filament.get_filelist_url(year, dayofyear)

KeyboardInterrupt: 

In [8]:
filelist2 = sorted(glob.glob(os.path.join("/data/Wind/ASCAT/GranCanaria/", "*nc*")))
filelist2

['/data/Wind/ASCAT/GranCanaria/ascat_20190710_094200_metopb_35329_eps_o_coa_3201_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/GranCanaria/ascat_20190710_100900_metopa_66016_eps_o_coa_3201_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/GranCanaria/ascat_20190710_213300_metopb_35336_eps_o_coa_3201_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/GranCanaria/ascat_20190710_215700_metopa_66023_eps_o_coa_3201_ovw.l2.nc.gz']

In [12]:
filelistclean = []
for ifile, dataurl in enumerate(filelist2):

    logger.info("Working on {}".format(dataurl))
    wind = filament.Wind()
    res = wind.read_ascat(dataurl, vis.extent)
    
    if res:
        filelistclean.append(dataurl)
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("sentine2-", dataurl.split("/")[-1].split(".")[0], ".jpg"))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(111, projection=myproj)
        vis.add_to_plot(ax, myproj)
        wind.add_to_plot(fig, ax, domain=(-16.75, -16., 27., 28.4), cmap=plt.cm.hot,
                         clim=[0, 12], cbarloc=[0.6, 0.25, 0.25, 0.015],
                         vis=True,
                         date=wind.time.strftime("%Y-%m-%d %H:%M"))
        filament.decorate_map(ax, (-16.5, -15.1, 27.1, 28.4), 
                              np.arange(-18., -13., 0.2), 
                              np.arange(26., 29.1, 0.2))
        # plt.show()
        plt.savefig(figname, dpi=300, bbox_inches="tight")
        plt.close()

INFO:root:Working on /data/Wind/ASCAT/GranCanaria/ascat_20190710_094200_metopb_35329_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as sentine2-ascat_20190710_094200_metopb_35329_eps_o_coa_3201_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/GranCanaria/ascat_20190710_100900_metopa_66016_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as sentine2-ascat_20190710_100900_metopa_66016_eps_o_coa_3201_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/GranCanaria/ascat_20190710_213300_metopb_35336_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as sentine2-ascat_20190710_213300_metopb_35336_eps_o_coa_3201_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/GranCanaria/ascat_20190710_215700_metopa_66023_eps_o_coa_3201_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as sentine2-ascat_20190710_215700_

In [56]:
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [57]:
wind = filament.Wind()
res = wind.read_ascat(dataurl, vis.extent)

INFO:Filament:Subsetting data to region of interest


In [60]:
ww = wind.time

In [61]:
ww

datetime.datetime(2019, 7, 10, 9, 42)